In [1]:
import ast

In [14]:
expr = '''
import threading
from functools import wraps, update_wrapper
import logging
from uuid import uuid4

from django.shortcuts import render_to_response
from pymongo.errors import PyMongoError
from redis import RedisError
from sqlalchemy.exc import SQLAlchemyError
from thriftpy.thrift import TException
from django.views.generic import View
from common.conf import conf
from common.mongo.error_charge_config import get_charge_person
from common.mysql_session import DbReadSession, CustomerDbSession
from common.send_email import send_text_email

from common.log import threading_local
from common.station_db_session import print_sql
from website.station.dals.station_customer_models import Region

from website.utils.views import BasePermitView, FormAjaxResponseMixin
from website.utils.views import logout_view

from service.dals.fresh_station import get_fresh_station
from service.station import StationRole

from datetime import datetime, timedelta
from common.mysql.models.user_station import StationUser, StationPartner
from website.utils.param_check import GmError, ErrCode, ParamChecker
from website.utils.utf8 import set_utf8
from station.models import CoreUser
from urllib.parse import unquote
from django.http.response import JsonResponse, HttpResponse

logger = logging.getLogger('station.view.common')
app_logger = logging.getLogger('app')
new_access_logger = logging.getLogger('new_access')
access_logger = logging.getLogger('access')
error_email_logger = logging.getLogger('error_email')


class MockStation():
    name = '默认模拟站点'
    about = '登录用户尚未关联任何站点.'
    salemenu = {}
    visible_salemenu = {}

    def __getitem__(self, name):
        return getattr(self, name)

    def __contains__(self, key):
        return hasattr(self, key)


mockStationInstance = MockStation()


def get_no_station(request, *args, **kwargs):
    logout_view(request, *args, **kwargs)
    return render_to_response('logout.html', {'amity_message': '当前帐号无可管理站点，请联系系统管理员！'})


app_name = 'station'


class ParamCheckMixin:
    # 给view混入

    param_check_dict = dict()

    def _param_check(self):
        self.params = {}
        for k, v in self.param_check_dict.items():
            arg_dict = {
                'param_map': self.request_param,
                'key': k,
                'min_': v.min_,
                'max_': v.max_,
                'opt': v.optional,
                'default': v.default,
                'edit': v.edit,
                'allow_space': v.allow_space,
            }
            param_type = v.param_type
            if hasattr(ParamChecker, 'check_' + param_type.__name__):
                checker = getattr(ParamChecker, 'check_' + param_type.__name__)
                param = checker(**arg_dict)
            else:  # 自定义函数做校验
                if k in self.request_param and self.request_param[k] != '':
                    param = param_type(self.request_param[k])
                elif v.optional:
                    param = v.default
                else:
                    raise GmError(ErrCode.param_err, '缺少{0}参数'.format(k))
            if param is not None:
                self.params[k] = param


class NoLoginView(View, ParamCheckMixin, FormAjaxResponseMixin):
    def __init__(self, request, *args, **kwargs):
        # 每个新请求创建时清空线程全局变量
        threading_local.request = {}
        threading_local.response = {}

        request.request_time = datetime.now()
        try:
            if request.method == 'POST':
                if hasattr(request, 'FILES') and request.FILES:
                    # 有文件上传的话，request.body.decode() 会报 UnicodeDecodeError
                    self.param_str = "FILE FOUND " + request.POST.urlencode()
                elif request.META.get('QUERY_STRING'):
                    self.param_str = "{}|{}".format(request.META['QUERY_STRING'], unquote(request.body.decode()))
                else:
                    self.param_str = unquote(request.body.decode())
            else:
                self.param_str = request.META['QUERY_STRING']
        except Exception:
            self.param_str = '*BODY CAN NOT DECODE*'

        self.__set_request_local(request)
        app_logger.info(
            '<<START>>{},{},{},{}'.format(
                threading_local.request['request_path'],
                threading_local.request['request_method'],
                threading_local.request['param_string_length'],
                self.param_str,
            )
        )  # 这是同一个请求在 app log 当中打的数条日志中的第一条日志

        super().__init__(**kwargs)
        self.request_method = request.method.lower()
        if self.request_method == 'post':
            self.request_param = request.POST.dict()
        elif self.request_method == 'get':
            self.request_param = request.GET.dict()
        self.__set_request_local(request)  # self.context 中有数据后，再设置一下 threading_local.request

    @classmethod
    def as_view(cls, **initkwargs):
        def view(request, *args, **kwargs):
            self = cls(request=request, **initkwargs)
            self.args = args
            self.kwargs = kwargs
            return self.dispatch(request, *args, **kwargs)

        view.view_class = cls
        view.view_initkwargs = initkwargs

        update_wrapper(view, cls)
        return view

    def __set_request_local(self, request):
        """在 threading_local.request 这个线程全局变量中储存 request 相关的数据，用于 log 的输出"""
        # 为了一个请求多次调用 __set_request_local 没有副作用，使用了下面的写法
        threading_local.request['request_uid'] = (
            threading_local.request.get('request_uid') or request.META.get('HTTP_X_GUANMAI_REQUEST_ID') or uuid4().hex
        )
        threading_local.request['request_path'] = request.path
        threading_local.request['request_method'] = request.method
        threading_local.request['param_string_length'] = len(self.param_str)
        threading_local.request['param_string_striped_by_max_length'] = self.param_str[:int(conf.access_log_max_length)]

    def __set_response_local(
            self, response_obj, *, ret_body=None, ret_body_length=None, ret_code=None, ret_msg=None, extra_value=None
    ):
        threading_local.response['status'] = response_obj.status_code
        if ret_body is not None:
            threading_local.response['ret_body_length'] = ret_body_length if ret_body_length is not None else len(ret_body)
            threading_local.response['ret_body_striped_by_max_length'] = ret_body[:int(conf.access_log_max_length)]
            # ret_body_length 和 ret_body_striped_by_max_length 可以对不上。当 ret_body 是 excel 时，ret_body_length 是这个
            # 二进制文件的长度，ret_body_striped_by_max_length 只是一个很短的提示字符串
        if ret_code is not None:
            threading_local.response['ret_code'] = ret_code
        if ret_msg is not None:
            threading_local.response['ret_msg'] = ret_msg
        if extra_value is not None:
            threading_local.response['extra_value'] = extra_value  # extra 用于储存一些临时字段

    def monitor_deal(self, request, response_obj, exception_str='', status=200,
                     exception=False, error_code=0):
        # 打印access日志，发送报错邮件
        now_time = datetime.now()
        deal_time = int(((now_time - request.request_time).total_seconds()) * 1000)
        threading_local.request['deal_time'] = str(deal_time)
        total_format = ''
        error_str = ''
        try:
            request_format_one = '[{0}||{1}]'.format(
                request.path, request.method
            )
            if isinstance(response_obj, JsonResponse) and response_obj.content:
                status = response_obj.status_code
                # JsonResponse 类型
                data_str = response_obj.content.decode()
                data = json.loads(data_str)
                try:
                    error_code = data.get('code', 0)
                    error_str = str(data.get('msg', ''))[:64]
                    r_data = str(data.get('data', ''))[:512]
                except AttributeError:  # data 可能是一个 list
                    error_code = -1
                    error_str = ''
                    r_data = ''
                    logger.warning("Found a JsonResponse instance with no code, msg or data. path=" + request.path)

                request_format_two = '[{0}||{1}||{2}||{3}][{4}]'.format(
                    status, error_code, error_str, r_data, self.param_str[:512]
                )
                total_format = '{}{}'.format(request_format_one, request_format_two)
                logger.info(total_format)
                self.__set_response_local(response_obj, ret_body=data_str, ret_code=error_code, ret_msg=error_str)

            elif isinstance(response_obj, HttpResponse) and response_obj.content:
                status = response_obj.status_code
                # HttpResponse 类型
                if response_obj.has_header('Content-Type') and 'excel' in response_obj['Content-Type']:
                    data = "<download file>"
                else:
                    try:
                        data = response_obj.content.decode().replace('\n', ' ').replace('\r\n', ' ').replace('\r', ' ')
                    except UnicodeDecodeError as e:
                        # 如果这里报错了，可能的原因：响应是二进制文件。在上面增加这种类型的文件的 Content-Type 可以修复这个问题。
                        logger.exception(e)
                        data = "<UnicodeDecodeError>"

                # -998定义的模板标识，和监控脚本对应
                error_code = -998
                error_str = ''
                request_format_two = '[{0}||{1}||{2}||{3}][{4}]'.format(
                    status, error_code, error_str, data[:512], self.param_str[:512]
                )
                total_format = '{}{}'.format(request_format_one, request_format_two)
                logger.info(total_format)
                self.__set_response_local(response_obj, ret_body=data, ret_body_length=len(response_obj.content))

            elif exception:
                # -999定义的未知错误，和监控脚本对应
                error_code = error_code if error_code else -999
                error_str = exception_str[:256]
                request_format_two = '[{0}||{1}||{2}||{3}][{4}]'.format(
                    status, error_code, error_str, '', self.param_str[:512]
                )
                total_format = '{}{}'.format(request_format_one, request_format_two)
                logger.info(total_format)

        except Exception as e:
            logger.exception(str(e))


    @set_utf8
    def dispatch(self, request, *args, **kwargs):
        self.param_str = self.param_str.replace('\n', ' ').replace('\r\n', ' ').replace('\r', ' ')
        response_obj = None
        is_send = False
        try:
            self._param_check()
            # if self.params['token'] != '292z599a0ecf11e8bb67w20ms2db51304ff1idua0ecf11e88246720h02db5130':
            #     raise GmError(ErrCode.param_err, 'token错误')

            response_obj = super().dispatch(request, *args, **kwargs)
            self.monitor_deal(request, response_obj)
        except GmError as e:
            logger.exception(str(e))
            self.monitor_deal(request, response_obj, exception_str=str(e), exception=True, error_code=e.err_code)
            response_obj = self.JsonErrorResponse(str(e), code=e.err_code)
            self.__set_response_local(response_obj, ret_code=e.err_code, ret_msg=str(e))
        # thrift错误
        except TException as e:
            logger.exception(str(e))
            is_send=True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse(str(e))
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        # mysql错误
        except SQLAlchemyError as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse(str(e))
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        # mongodb错误
        except PyMongoError as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse(str(e))
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        # Redis错误
        except RedisError as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse(str(e))
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        except Exception as e:
            logger.exception(str(e))
            server_error = 'SERVER ERROR: ' + str(e)
            self.monitor_deal(request, response_obj, exception_str=server_error, exception=True)
            response_obj = self.JsonErrorResponse(server_error)
            self.__set_response_local(response_obj, ret_code=1, ret_msg=server_error)

        try:
            # 这是同一个请求在 access log 当中打的唯一一条日志
            new_access_logger.info('')
            # 测试环境不发邮件，非2xx错误或者exception发送邮件
            if conf.get("test_env") == 'no' and is_send:
                error_email_logger.info('')
            # 这是同一个请求在 app log 当中打的数条日志中的最后一条日志
            app_logger.info(
                '<<END>>{},{}'.format(
                    threading_local.response.get('ret_body_length', 0),
                    threading_local.response.get('ret_body_striped_by_max_length', ''),
                    # 因为 conf.app_log_max_length < conf.access_log_max_length,
                    # 而且 threading_local.response['ret_body_striped_by_max_length'] 是
                    # 使用 access_log_max_length 截断的，所以这里使用截断的 ret_body_striped_by_max_length 没有问题
                )
            )
        except Exception as e:
            app_logger.info('<<END>>{},{}'.format(0, ''))
            logger.exception(str(e))
        return response_obj


class CommonBaseView(BasePermitView, ParamCheckMixin, FormAjaxResponseMixin):
    def monitor_deal(self, request, response_obj, exception_str='', status=200,
                     exception=False, error_code=0):
        # 打印access日志，发送报错邮件
        now_time = datetime.now()
        deal_time = int(((now_time - request.request_time).total_seconds()) * 1000)
        threading_local.request['deal_time'] = str(deal_time)
        total_format = ''
        error_str = ''
        try:
            request_format_one = '[{0}||{1}]'.format(
                request.path, request.method
            )
            if isinstance(response_obj, JsonResponse) and response_obj.content:
                status = response_obj.status_code
                # JsonResponse 类型
                data_str = response_obj.content.decode()
                data = json.loads(data_str)
                try:
                    error_code = data.get('code', 0)
                    error_str = str(data.get('msg', ''))[:64]
                    r_data = str(data.get('data', ''))[:512]
                except AttributeError:  # data 可能是一个 list
                    error_code = -1
                    error_str = ''
                    r_data = ''
                    logger.warning("Found a JsonResponse instance with no code, msg or data. path=" + request.path)

                request_format_two = '[{0}||{1}||{2}||{3}][{4}]'.format(
                    status, error_code, error_str, r_data, self.param_str[:512]
                )
                total_format = '{}{}'.format(request_format_one, request_format_two)
                logger.info(total_format)
                self.__set_response_local(response_obj, ret_body=data_str, ret_code=error_code, ret_msg=error_str)

            elif isinstance(response_obj, HttpResponse) and response_obj.content:
                status = response_obj.status_code
                # HttpResponse 类型
                if response_obj.has_header('Content-Type') and 'excel' in response_obj['Content-Type']:
                    data = "<download file>"
                else:
                    try:
                        data = response_obj.content.decode().replace('\n', ' ').replace('\r\n', ' ').replace('\r', ' ')
                    except UnicodeDecodeError as e:
                        # 如果这里报错了，可能的原因：响应是二进制文件。在上面增加这种类型的文件的 Content-Type 可以修复这个问题。
                        logger.exception(e)
                        data = "<UnicodeDecodeError>"

                # -998定义的模板标识，和监控脚本对应
                error_code = -998
                error_str = ''
                request_format_two = '[{0}||{1}||{2}||{3}][{4}]'.format(
                    status, error_code, error_str, data[:512], self.param_str[:512]
                )
                total_format = '{}{}'.format(request_format_one, request_format_two)
                logger.info(total_format)
                self.__set_response_local(response_obj, ret_body=data, ret_body_length=len(response_obj.content))

            elif exception:
                # -999定义的未知错误，和监控脚本对应
                error_code = error_code if error_code else -999
                error_str = exception_str[:256]
                request_format_two = '[{0}||{1}||{2}||{3}][{4}]'.format(
                    status, error_code, error_str, '', self.param_str[:512]
                )
                total_format = '{}{}'.format(request_format_one, request_format_two)
                logger.info(total_format)

        except Exception as e:
            logger.exception(str(e))


    @set_utf8
    def dispatch(self, request, *args, **kwargs):
        if str(self.group_id) in conf.pay_first_gray:return self.JsonErrorResponse(msg='系统正在升级中，请耐心等待...')     # 先款后货刷数据时需要关闭操作。
        self.param_str = self.param_str.replace('\n', ' ').replace('\r\n', ' ').replace('\r', ' ')
        response_obj = None
        is_send = False
        try:
            self._param_check()
            response_obj = super().dispatch(request, *args, **kwargs)
            self.monitor_deal(request, response_obj)
        except GmError as e:
            logger.exception(str(e))
            self.monitor_deal(request, response_obj, exception_str=str(e), exception=True, error_code=e.err_code)
            response_obj = self.JsonErrorResponse(str(e), code=e.err_code, data=e.data)
            self.__set_response_local(response_obj, ret_code=e.err_code, ret_msg=str(e))
        # thrift错误
        except TException as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse('系统繁忙，稍后再试')
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        # mysql错误
        except SQLAlchemyError as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse('系统繁忙，稍后再试')
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        # mongodb错误
        except PyMongoError as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True, )
            response_obj = self.JsonErrorResponse('系统繁忙，稍后再试')
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        # Redis错误
        except RedisError as e:
            logger.exception(str(e))
            is_send = True
            self.monitor_deal(request, response_obj,
                              exception_str=str(e), exception=True)
            response_obj = self.JsonErrorResponse('系统繁忙，稍后再试')
            self.__set_response_local(response_obj, ret_code=1, ret_msg=str(e))
        except Exception as e:
            logger.exception(str(e))
            server_error = 'SERVER ERROR: ' + str(e)
            self.monitor_deal(request, response_obj, exception_str=server_error, exception=True)
            try:
                msg=str(e)
                # 与任务相关提示
                if msg.startswith('任务'):
                    response_obj = self.JsonErrorResponse('已有同类任务正在执行，待完成后再添加')
                else:
                    response_obj = self.JsonErrorResponse('系统繁忙，稍后再试')
            except:
                response_obj = self.JsonErrorResponse('系统繁忙，稍后再试')
            self.__set_response_local(response_obj, ret_code=1, ret_msg=server_error)

        try:
            # 这是同一个请求在 access log 当中打的唯一一条日志
            new_access_logger.info('')
            # 测试环境不发邮件，非2xx错误或者exception发送邮件
            if conf.get("test_env") == 'no' and is_send:
                error_email_logger.info('')
            # 这是同一个请求在 app log 当中打的数条日志中的最后一条日志
            app_logger.info(
                '<<END>>{},{}'.format(
                    threading_local.response.get('ret_body_length', 0),
                    threading_local.response.get('ret_body_striped_by_max_length', ''),
                    # 因为 conf.app_log_max_length < conf.access_log_max_length, 而且 threading_local.response['ret_body_striped_by_max_length'] 是
                    # 使用 access_log_max_length 截断的，所以这里使用截断的 ret_body_striped_by_max_length 没有问题
                )
            )
        except Exception as e:
            app_logger.info('<<END>>{},{}'.format(0, ''))
            logger.exception(str(e))
        return response_obj

    def __set_request_local(self, request):
        """在 threading_local.request 这个线程全局变量中储存 request 相关的数据，用于 log 的输出"""
        # 为了一个请求多次调用 __set_request_local 没有副作用，使用了下面的写法
        threading_local.request['request_uid'] = (
            threading_local.request.get('request_uid') or request.META.get('HTTP_X_GUANMAI_REQUEST_ID') or uuid4().hex
        )
        threading_local.request['group_id'] = self.context.get('partner_id', '-')
        threading_local.request['station_id'] = self.context.get('station_id', '-')
        threading_local.request['username'] = request.user.username
        threading_local.request['userid'] = request.user.id
        threading_local.request['request_path'] = request.path
        threading_local.request['request_method'] = request.method
        threading_local.request['param_string_length'] = len(self.param_str)
        threading_local.request['param_string_striped_by_max_length'] = self.param_str[:int(conf.access_log_max_length)]

    def __set_response_local(
            self, response_obj, *, ret_body=None, ret_body_length=None, ret_code=None, ret_msg=None, extra_value=None
    ):
        threading_local.response['status'] = response_obj.status_code
        if ret_body is not None:
            threading_local.response['ret_body_length'] = ret_body_length if ret_body_length is not None else len(ret_body)
            threading_local.response['ret_body_striped_by_max_length'] = ret_body[:int(conf.access_log_max_length)]
            # ret_body_length 和 ret_body_striped_by_max_length 可以对不上。当 ret_body 是 excel 时，ret_body_length 是这个
            # 二进制文件的长度，ret_body_striped_by_max_length 只是一个很短的提示字符串
        if ret_code is not None:
            threading_local.response['ret_code'] = ret_code
        if ret_msg is not None:
            threading_local.response['ret_msg'] = ret_msg
        if extra_value is not None:
            threading_local.response['extra_value'] = extra_value  # extra 用于储存一些临时字段

    def __init__(self, request, *args, **kwargs):
        # 每个新请求创建时清空线程全局变量
        threading_local.request = {}
        threading_local.response = {}

        request.request_time = datetime.now()
        try:
            if request.method == 'POST':
                if hasattr(request, 'FILES') and request.FILES:
                    self.param_str = "FILE FOUND " + request.POST.urlencode()  # 有文件上传的话，request.body.decode() 会报 UnicodeDecodeError
                elif request.META.get('QUERY_STRING'):
                    self.param_str = "{}|{}".format(request.META['QUERY_STRING'], unquote(request.body.decode()))
                else:
                    self.param_str = unquote(request.body.decode())
            else:
                self.param_str = request.META['QUERY_STRING']
        except:
            self.param_str = '*BODY CAN NOT DECODE*'

        self.context = {}  # 这里给 self.context 赋值是因为 self.context 不存在的话 __set_request_local 会报错
        self.__set_request_local(request)
        app_logger.info(
            '<<START>>{},{},{},{}'.format(
                threading_local.request['request_path'],
                threading_local.request['request_method'],
                threading_local.request['param_string_length'],
                self.param_str,
            )
        )  # 这是同一个请求在 app log 当中打的数条日志中的第一条日志

        super().__init__(request, *args, **kwargs)
        if self.request_method == 'post':
            self.request_param = request.POST.dict()
        elif self.request_method == 'get':
            self.request_param = request.GET.dict()
        # TODO 优化，station上的字段应该做lazy loading,并不是每个请求都需要所有数据的

        dbsess = DbReadSession()

        station_id = dbsess.query(StationUser.station_id).filter(StationUser.user_id == request.user.id).scalar()

        t = datetime.now()
        today_begin = t - timedelta(hours=t.hour, minutes=t.minute, seconds=t.second)
        tomorrow_begin = today_begin + timedelta(days=1)

        if station_id is None:
            self.get = get_no_station
            self.post = get_no_station
            self.context = {
                'today_begin_timestamp': int(today_begin.timestamp()),
                'tomorrow_begin_timestamp': int(tomorrow_begin.timestamp())
            }
        else:
            self.station_id = station_id
            self.station = get_fresh_station(_id=self.station_id)  # or mockStationInstance
            self.userid = request.user.id
            self.username = request.user.username
            self.group_id = self.get_group_id()
            # 每次连接时,即设置当前城市
            setattr(request.user, 'station_id', self.station_id)
            setattr(request.user, 'station_name', self.station['name'])
            setattr(request.user, 'permissions', self.user_permission)
            setattr(request.user, 'group_id', self.group_id),
            setattr(request.user, 'distribute_cities', self.get_distribute_city_names())
            setattr(request.user, 'stock_method', self.station.get('stock_method') if self.station.get(
                'stock_method') else 1)

            user_type_iter = dbsess.query(CoreUser.type_id).filter(CoreUser.id == request.user.id).first()

            self.context = {
                'station_id': self.station_id,
                'is_staff': request.user.is_staff,
                'is_superman': request.user.is_superuser,
                'is_center_seller': self.station.get('role') == StationRole.Seller.value,
                'role': self.station.get('role'),
                'type': user_type_iter[0],
                'today_begin_timestamp': int(today_begin.timestamp()),
                'tomorrow_begin_timestamp': int(tomorrow_begin.timestamp()),
                'is_fqt': 1 if self.station_id == 'T847' else 0,
                'partner_id': self.group_id or 0,
                'show_sku_outer_id': self.station.get('show_sku_outer_id', 0),
                'default_settle_way': self.station.get('default_settle_way', 1),
                'show_tax_rate': self.station.get('show_tax_rate', 0)
            }
        self.__set_request_local(request)  # self.context 中有数据后，再设置一下 threading_local.request

    def get_group_id(self):
        return self.station.get('partner_id', 0)

    def get_context(self, context=None):
        if context:
            self.context.update(context)
        return self.context

    # 检查是否存在时间限制
    def check_datatime_limit(self):
        station_ids = [
            'T001', 'T002', 'T166', 'T167', 'T207', 'T208',
            'T209', 'T210', 'T211', 'T212', 'T213', 'T214',
            'T215', 'T216', 'T217', 'T218', 'T219', 'T220',
            'T221', 'T222', 'T223', 'T224', 'T296', 'T353',
            'T354', 'T355', 'T356', 'T357', 'T358', 'T715']
        cur_time = datetime.now()
        cur_hour = cur_time.hour
        if self.station_id in station_ids and (cur_hour < 6 or cur_hour >= 23):
            return 1
        return 0

    def get_distribute_city_names(self):
        dbsess = CustomerDbSession()
        res = dbsess.query(Region.code, Region.name).filter(Region.code.in_(self.station['distribute_cities'])).all()
        city_dict = {city_tpl[0]: city_tpl[1] for city_tpl in res}
        return [city_dict[code] for code in reversed(self.station['distribute_cities'])]


import json
from django.views.generic import View


class TypeInvalidException(Exception):
    def __init__(self):
        super(TypeInvalidException, self).__init__('指定校验的参数类型有误')


class InvalidParamException(Exception):
    pass


# 返回目标字典中仅包含指定keys的子集
def dict_proj(d, keys):
    return {k: v for k, v in d.items() if k in keys}


# 去掉值为None的目标字典的子集
def dict_exclude_none(d):
    return {k: v for k, v in d.items() if v is not None}


# 按照required或者option定义的参数限制条件来格式化参数字典
# 限制条件形如: {'price': int}
def format_params(param_dict, required=None, optional=None):
    def format_value(p_type, value):
        if p_type is bool:
            return bool(int(value))
        elif p_type == list:
            return list(json.loads(value))
        elif p_type == 'json':
            return json.loads(value)
        elif isinstance(p_type, type):
            return p_type(value)
        raise TypeInvalidException()

    formated_param_dict = {}
    if required:
        for name, param_type in required.items():
            if not name in param_dict:
                raise InvalidParamException("%s is required" % name)
            try:
                f_value = format_value(param_type, param_dict[name])
            except TypeInvalidException as e:
                continue
            except Exception as e:
                raise InvalidParamException("%s ‘s type is wrong" % name)
            formated_param_dict.update({name: f_value})

    if optional:
        for name, param_type in optional.items():
            if name in param_dict:
                try:
                    f_value = format_value(param_type, param_dict[name])
                except TypeInvalidException as e:
                    continue
                except Exception as e:
                    raise InvalidParamException("%s ‘s type is wrong" % name)
                formated_param_dict.update({name: f_value})
    return formated_param_dict


class FormatParamsBaseView(View, FormAjaxResponseMixin):
    required_params = {}
    optional_params = {}

    def dispatch(self, request, *args, **kwargs):
        try:
            params = format_params(getattr(request, request.method).dict(), self.required_params, self.optional_params)
        except Exception as e:
            return self.JsonErrorResponse(str(e))
        kwargs.update(params)
        return super(FormatParamsBaseView, self).dispatch(request, *args, **kwargs)


def exception_resp(func):
    @wraps(func)
    def wrapper(self, *args, **kwargs):
        try:
            return func(self, *args, **kwargs)
        except Exception as e:
            logger.exception(str(e))
            return self.JsonErrorResponse(str(e))

    return wrapper


def log_allocate(url, method, error_code, error_str):
    # 按照上面参数匹配错误记录表，查找对应的负责人
    # 过滤掉缺少参数的
    if '缺少' in error_str:
        return []
    query = {
        'url': url,
        'method': method.upper(),
        'error_code': error_code,
        'error_str': {'$regex': error_str[:50]}
    }
    query_data = get_charge_person(query, ['email'])
    email_list = list({i['email'] for i in query_data if 'email' in i})
    # 'not_send@gmail.cn' 标识不会发送邮件，和配置发送邮件的地方对应
    if len(email_list) == 1 and email_list[0] == 'not_send@gmail.cn':
        mail_list = []
    elif not email_list:
        mail_list = ['g_dev@xiaonongnv.com']
    else:
        mail_list = email_list
    return mail_list

'''

In [15]:
expr_ast = ast.parse(expr)
expr_ast

SyntaxError: EOL while scanning string literal (<unknown>, line 221)

In [12]:
ast.dump(expr_ast)

"Module(body=[ImportFrom(module='station.views.common', names=[alias(name='CommonBaseView', asname=None)], level=0), ImportFrom(module='django.utils.translation', names=[alias(name='gettext', asname='_')], level=0), ClassDef(name='TestView', bases=[Name(id='CommonBaseView', ctx=Load())], keywords=[], body=[FunctionDef(name='get', args=arguments(args=[arg(arg='self', annotation=None), arg(arg='request', annotation=None)], vararg=arg(arg='args', annotation=None), kwonlyargs=[], kw_defaults=[], kwarg=arg(arg='kwargs', annotation=None), defaults=[]), body=[Return(value=Call(func=Attribute(value=Name(id='self', ctx=Load()), attr='JsonErrorResponse', ctx=Load()), args=[Call(func=Attribute(value=Str(s='请求参数有误 {}'), attr='format', ctx=Load()), args=[Str(s='a != b')], keywords=[])], keywords=[]))], decorator_list=[], returns=None)], decorator_list=[])])"

In [8]:
class CrazyTransformer(ast.NodeTransformer):

    def visit_BinOp(self, node):
        print(node.__dict__)
        node.op = ast.Mult()
        print(node.__dict__)
        return node

In [9]:
transformer = CrazyTransformer()
transformer.visit(expr_ast)

{'left': <_ast.Name object at 0x105362d30>, 'op': <_ast.Add object at 0x103769470>, 'right': <_ast.Name object at 0x105362d68>, 'lineno': 3, 'col_offset': 11}
{'left': <_ast.Name object at 0x105362d30>, 'op': <_ast.Mult object at 0x1054024a8>, 'right': <_ast.Name object at 0x105362d68>, 'lineno': 3, 'col_offset': 11}
